## Auto ML com a biblioteca H2O

In [2]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator
import pandas as pd

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,19 mins 01 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_Luiz_Philipe_Rangel_cp4p9h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.957 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [4]:
#Importa dados 
imp = pd.read_csv('C:/Users/Luiz Philipe Rangel/Documents/GitHub/Auto_ML/Churn_treino.csv', sep=";")
imp.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,10134888,1
1,608,Spain,Female,41,1,8380786,1,0,1,11254258,0
2,502,France,Female,42,8,1596608,3,1,0,11393157,1
3,699,France,Female,39,1,0,2,0,0,9382663,0
4,850,Spain,Female,43,2,12551082,1,1,1,790841,0


In [5]:
# divide em treino e teste
imp = h2o.H2OFrame(imp)
treino,teste = imp.split_frame(ratios=[.7])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
#Transforma a variável dependente em fator
treino["Exited"] = treino["Exited"].asfactor()
teste["Exited"] = teste["Exited"].asfactor()

In [7]:
#Auto ML
#Busca o modelo por 180 segundos, podemos em vez disso definir max_models
modelo_automl = H2OAutoML(max_runtime_secs=180, sort_metric = 'AUC')
modelo_automl.train( y="Exited", training_frame=treino)

AutoML progress: |█
13:50:36.139: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),10/42
# GBM base models (used / total),7/33
# DeepLearning base models (used / total),3/6
# DRF base models (used / total),0/2
# GLM base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [8]:
#Random Forest
model_floresta = H2ORandomForestEstimator(nfolds=5)
model_floresta.train( y="Exited", training_frame=treino)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1678984225374_5373


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          544537                 19           20           19.98         801           929           862

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.11157470267123677
RMSE: 0.3340279968374459
LogLoss: 0.7424321131087919
Mean Per-Class Error: 0.25153041648855906
AUC: 0.826008389512337
AUCPR: 0.6514245336041573
Gini: 0.652016779024674

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.35294117647058815
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      4915  613   0.1109   (613.0/5528.0)
1      551   854   0.3922   (551.0/1405.0)
Total  5466  1467  0.1679   (1164.0/6933.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.352941     0.594708  183
max f2                       0.195911     0.656466  257
max f0point5                 0.608696     0.664271  97
max accuracy                 0.5625       0.858214  109
max precision                0.954545     0.955056  6
max recall                   0            1         399
max specificity              1            0.999276  0
max absolute_mcc             0.529412     0.509297  120
max min_per_class_accuracy   0.209782     0.745907  249
max mean_per_class_accuracy  0.272702     0.759354  220
max tns                      1            5524      0
max fns                      1            1333      0
max fps                      0            5528      399
max tps                      0            1405      399
max tnr                      1            0.999276  0
max fnr                      1            0.948754  0
max fpr                      0            1         399
max tpr                      0            1         399

Gains/Lift Table: Avg response rate: 20,27 %, avg score: 21,03 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0109621                   1                  4.67481   4.67481            0.947368         1           0.947368                    1                   0.0512456       0.0512456                  367.481   367.481            0.050522
2        0.0204818                   0.9375             4.56069   4.62177            0.924242         0.94843     0.93662                     0.976031            0.0434164       0.0946619                  356.069   362.177            0.0930338
3        0.0310111                   0.888889           4.25856   4.49845            0.863014         0.909944    0.911628                    0.953592            0.0448399       0.139502                   325.856   349.845            0.136065
4        0.0411077                   0.857143           4.51156   4.50167            0.914286         0.870099    0.912281                    0.933085            0.0455516       0.185053                   351.156   350.167            0.180531
5        0.0500505                   0.816746           4.13863   4.4368             0.83871          0.834097    0.899135                    0.9

In [9]:
#Ranking dos melhores AutoML
ranking = modelo_automl.leaderboard
ranking = ranking.as_data_frame()
ranking

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
0,StackedEnsemble_AllModels_3_AutoML_1_20230316_...,0.863501,0.333179,0.704982,0.234705,0.317490,0.100800
1,StackedEnsemble_AllModels_4_AutoML_1_20230316_...,0.863428,0.333105,0.705480,0.234784,0.317342,0.100706
2,GBM_grid_1_AutoML_1_20230316_135036_model_12,0.862864,0.336394,0.701271,0.235508,0.318835,0.101656
3,StackedEnsemble_BestOfFamily_4_AutoML_1_202303...,0.862665,0.334369,0.702720,0.230602,0.318123,0.101202
4,GBM_grid_1_AutoML_1_20230316_135036_model_27,0.861960,0.338108,0.699947,0.241040,0.319758,0.102245
5,StackedEnsemble_AllModels_2_AutoML_1_20230316_...,0.860778,0.335441,0.701928,0.227290,0.318463,0.101419
6,GBM_grid_1_AutoML_1_20230316_135036_model_10,0.859964,0.337840,0.698524,0.233528,0.319440,0.102042
7,GBM_grid_1_AutoML_1_20230316_135036_model_21,0.859848,0.337539,0.695947,0.228520,0.319272,0.101935
8,StackedEnsemble_BestOfFamily_3_AutoML_1_202303...,0.859561,0.336464,0.700464,0.237727,0.318711,0.101577
9,StackedEnsemble_AllModels_1_AutoML_1_20230316_...,0.858136,0.338635,0.695462,0.235387,0.319757,0.102245


In [10]:
performance = model_floresta.model_performance(test_data=teste)
performance

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.10783068127086756
RMSE: 0.3283758232130794
LogLoss: 0.47365972252602584
Mean Per-Class Error: 0.2284111584747745
AUC: 0.8488608894549423
AUCPR: 0.6640487717378195
Gini: 0.6977217789098846

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3333333331346512
       0     1    Error    Rate
-----  ----  ---  -------  --------------
0      2151  284  0.1166   (284.0/2435.0)
1      215   417  0.3402   (215.0/632.0)
Total  2366  701  0.1627   (499.0/3067.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.333333     0.625656  121
max f2                       0.153333     0.68779   230
max f0point5                 0.54         0.68038   48
max accuracy                 0.54         0.861428  48
max precision                0.94         0.929825  3
max recall                   0            1         399
max specificity              1            0.998768  0
max absolute_mcc             0.445        0.536308  72
max min_per_class_accuracy   0.22         0.762658  178
max mean_per_class_accuracy  0.321818     0.77232   127
max tns                      1            2432      0
max fns                      1            628       0
max fps                      0            2435      399
max tps                      0            632       399
max tnr                      1            0.998768  0
max fnr                      1            0.993671  0
max fpr                      0            1         399
max tpr                      0            1         399

Gains/Lift Table: Avg response rate: 20,61 %, avg score: 21,93 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0130421                   0.96               4.36756   4.36756            0.9              0.9735       0.9                         0.9735              0.056962        0.056962                   336.756   336.756            0.0553193
2        0.0247799                   0.92               4.58325   4.46973            0.944444         0.929444     0.921053                    0.952632            0.0537975       0.110759                   358.325   346.973            0.108295
3        0.0299967                   0.880133           4.24624   4.43086            0.875            0.898333     0.913043                    0.943188            0.0221519       0.132911                   324.624   343.086            0.129626
4        0.0427127                   0.86               3.98182   4.29718            0.820513         0.868974     0.885496                    0.921094            0.0506329       0.183544                   298.182   329.718            0.177384
5        0.0544506                   0.82               4.85285   4.41696            1                0.830889     0.91018                     0.901649            0.056962        0.240506                   385.285   341.696            0.234346
6        0.103032                    0.62               3.0941    3.79321            0.637584         0.707582     0.781646                    0.810143            0.150316        0.390823                   209.41    279.321            0.362486
7        0.15357                     0.48               2.75517   3.4516             0.567742         0.535782     0.711253                    0.719854            0.139241        0.530063                   175.517   245.16             0.474211
8        0.202478   

O mdelo do Auto ML teve a AUC melhor

In [11]:
#Prever 
teste = pd.read_csv('C:/Users/Luiz Philipe Rangel/Documents/GitHub/Auto_ML/Churn_prever.csv', sep=";")
teste = h2o.H2OFrame(teste)
prever = modelo_automl.leader.predict(teste)
prever

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.930471,0.069529
0,0.930543,0.0694568
1,0.00440279,0.995597
0,0.969339,0.0306607
0,0.937738,0.0622618
0,0.856324,0.143676
0,0.794603,0.205397
0,0.872466,0.127534
0,0.910059,0.0899405
